# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 8 2023 9:10AM,258926,10,37814,"Digital Brands, LLC",Released
1,Mar 8 2023 9:10AM,258926,10,37815,"Digital Brands, LLC",Released
2,Mar 8 2023 9:10AM,258926,10,37816,"Digital Brands, LLC",Released
3,Mar 8 2023 9:10AM,258926,10,37817,"Digital Brands, LLC",Released
4,Mar 8 2023 8:57AM,258925,10,MSP219267,"Methapharm, Inc.",Released
5,Mar 8 2023 8:57AM,258925,10,MSP219268,"Methapharm, Inc.",Released
6,Mar 8 2023 8:57AM,258925,10,MSP219269,"Methapharm, Inc.",Released
7,Mar 8 2023 8:57AM,258925,10,MSP219272,"Methapharm, Inc.",Released
8,Mar 8 2023 8:57AM,258925,10,MSP219281,"Methapharm, Inc.",Released
9,Mar 8 2023 8:57AM,258925,10,MSP219227,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,258922,Released,1
24,258923,Released,1
25,258924,Released,10
26,258925,Released,8
27,258926,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258922,NaN,NaN,1.0
258923,NaN,NaN,1.0
258924,NaN,NaN,10.0
258925,NaN,NaN,8.0
258926,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258811,12.0,1.0,2.0
258816,0.0,1.0,0.0
258828,0.0,0.0,1.0
258862,0.0,0.0,1.0
258886,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258811,12,1,2
258816,0,1,0
258828,0,0,1
258862,0,0,1
258886,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258811,12,1,2
1,258816,0,1,0
2,258828,0,0,1
3,258862,0,0,1
4,258886,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258811,12,1,2
1,258816,,1,
2,258828,,,1
3,258862,,,1
4,258886,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC"
4,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc."
12,Mar 8 2023 8:55AM,258924,12,Hush Hush
22,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC"
23,Mar 8 2023 8:52AM,258921,10,Bio-PRF
31,Mar 8 2023 8:51AM,258922,10,"Senseonics, Incorporated"
32,Mar 8 2023 8:47AM,258920,10,Emerginnova
34,Mar 8 2023 8:46AM,258919,19,Else Nutrition
35,Mar 8 2023 8:44AM,258918,12,Hush Hush
39,Mar 8 2023 8:43AM,258917,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",,,4
1,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",,,8
2,Mar 8 2023 8:55AM,258924,12,Hush Hush,,,10
3,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",,,1
4,Mar 8 2023 8:52AM,258921,10,Bio-PRF,,,8
5,Mar 8 2023 8:51AM,258922,10,"Senseonics, Incorporated",,,1
6,Mar 8 2023 8:47AM,258920,10,Emerginnova,,,2
7,Mar 8 2023 8:46AM,258919,19,Else Nutrition,,,1
8,Mar 8 2023 8:44AM,258918,12,Hush Hush,,,4
9,Mar 8 2023 8:43AM,258917,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",4,,
1,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",8,,
2,Mar 8 2023 8:55AM,258924,12,Hush Hush,10,,
3,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",1,,
4,Mar 8 2023 8:52AM,258921,10,Bio-PRF,8,,
5,Mar 8 2023 8:51AM,258922,10,"Senseonics, Incorporated",1,,
6,Mar 8 2023 8:47AM,258920,10,Emerginnova,2,,
7,Mar 8 2023 8:46AM,258919,19,Else Nutrition,1,,
8,Mar 8 2023 8:44AM,258918,12,Hush Hush,4,,
9,Mar 8 2023 8:43AM,258917,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",4,,
1,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",8,,
2,Mar 8 2023 8:55AM,258924,12,Hush Hush,10,,
3,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",1,,
4,Mar 8 2023 8:52AM,258921,10,Bio-PRF,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",4.0,NaN,NaN
1,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",8.0,NaN,NaN
2,Mar 8 2023 8:55AM,258924,12,Hush Hush,10.0,NaN,NaN
3,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",1.0,NaN,NaN
4,Mar 8 2023 8:52AM,258921,10,Bio-PRF,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",4.0,0.0,0.0
1,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",8.0,0.0,0.0
2,Mar 8 2023 8:55AM,258924,12,Hush Hush,10.0,0.0,0.0
3,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",1.0,0.0,0.0
4,Mar 8 2023 8:52AM,258921,10,Bio-PRF,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1812287,26.0,1.0,12.0
12,2847926,20.0,3.0,0.0
19,1035564,14.0,13.0,0.0
21,517809,2.0,0.0,0.0
22,258912,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1812287,26.0,1.0,12.0
1,12,2847926,20.0,3.0,0.0
2,19,1035564,14.0,13.0,0.0
3,21,517809,2.0,0.0,0.0
4,22,258912,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,26.0,1.0,12.0
1,12,20.0,3.0,0.0
2,19,14.0,13.0,0.0
3,21,2.0,0.0,0.0
4,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,26.0
1,12,Released,20.0
2,19,Released,14.0
3,21,Released,2.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,21,22
Status,,,,,
Completed,12.0,0.0,0.0,0.0,0.0
Executing,1.0,3.0,13.0,0.0,1.0
Released,26.0,20.0,14.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0
1,Executing,1.0,3.0,13.0,0.0,1.0
2,Released,26.0,20.0,14.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0
1,Executing,1.0,3.0,13.0,0.0,1.0
2,Released,26.0,20.0,14.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()